In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

In [3]:
df = pd.read_csv("train.csv")

In [4]:
dropped_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']

In [5]:
df.drop(dropped_cols, axis=1, inplace=True)

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
def model_score(model_generator, train_X, y, parameters={}):
    model = model_generator(**parameters)
    rmse = np.sqrt(-cross_val_score(model, train_X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [7]:
#log transform the target:
df["SalePrice"] = np.log1p(df["SalePrice"])

#log transform skewed numeric features:
numeric_feats = df.dtypes[df.dtypes != "object"].index

skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

df[skewed_feats] = np.log1p(df[skewed_feats])
#filling NA's with the mean of the column:
df = df.fillna(df.mean())
df = pd.get_dummies(df)

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
  {'C': [1, 10], 'kernel': ['linear']},
 ]
clf = GridSearchCV(svm.SVR(), param_grid)

In [10]:
y = df['SalePrice']

In [17]:
df.drop('SalePrice', axis=1, inplace=True)

In [22]:
clf = svm.SVR(C=10, kernel='linear')

In [12]:
model_score(svm.SVR, df, y, {"C":10, "kernel":'linear'})

KeyboardInterrupt: 